In [ ]:
#Assignment 3 - Part B: Practical Implementation & Deployment
#By Nanfuka Fatuma 
#B36106.

In [ ]:
file 3 _PROJECT: Optimizing Food Distribution Scheduling to Refugee Settlements in West Nile, Uganda.

In [ ]:
# constructing the training script
!pip install stable-baselines3
!pip install stable-baselines3[extra]
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

import sys
import os   
sys.path.append(os.getcwd())
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback

from src.env.food_env import FoodcenterEnv

print("Starting training...")

MODEL_DIR = os.path.join(os.getcwd(), "models")
os.makedirs(MODEL_DIR, exist_ok=True)


def make_env():
    return FoodcenterEnv(num_centers=4)


env = DummyVecEnv([make_env])

model = PPO('MlpPolicy', env, verbose=1, tensorboard_log="./tb_logs", learning_rate=3e-4, batch_size=2048)

# Callbacks
checkpoint_callback = CheckpointCallback(save_freq=10000, save_path=MODEL_DIR, name_prefix='ppo_food')
eval_env = DummyVecEnv([make_env])
eval_callback = EvalCallback(eval_env, best_model_save_path=MODEL_DIR,
                             log_path='logs/', eval_freq=5000, deterministic=True, render=False)

model.learn(total_timesteps=200000, callback=[checkpoint_callback, eval_callback])

model_path = os.path.join(MODEL_DIR, 'ppo_final_model.zip')
model.save(model_path)
print(f"Saved model to {model_path}")